In [17]:
import pandas as pd
import numpy as np
from pybaseball import batting_stats_range
from pybaseball import cache
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

cache.enable()

# Get batting stats for specified time range from PyBaseball
data = batting_stats_range(start_dt='2015-03-01', end_dt='2022-12-31')

C:\Users\Wyatt Baldwin\anaconda3\lib\site-packages\pybaseball\league_batting_stats.py:70: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  table = table.drop('', 1)


In [26]:
# Group data by Name
grouped_data = data.groupby(by='Name').mean()
grouped_data.reset_index(inplace=True)

In [32]:
print(grouped_data)

                               Name   Age   #days      G      PA      AB  \
0                      A.J. Burnett  38.0  2681.0   24.0    52.0    42.0   
1                         A.J. Cole  26.0  1760.0   19.0    30.0    26.0   
2                        A.J. Ellis  37.0  1588.0  240.0   759.0   646.0   
3                      A.J. Griffin  29.0  2006.0    3.0     8.0     7.0   
4                      A.J. Jimenez  27.0  1952.0    7.0    13.0    12.0   
...                             ...   ...     ...    ...     ...     ...   
2383                   Zack Littell  25.0   504.0    4.0     4.0     4.0   
2384                     Zack Short  27.0   158.0   67.0   197.0   165.0   
2385                   Zack Wheeler  31.0   494.0  102.0   226.0   208.0   
2386      \xc3\x81ngel Pag\xc3\xa1n  34.0  2316.0  262.0  1093.0  1006.0   
2387  \xc3\x93scar Hern\xc3\xa1ndez  26.0   917.0   25.0    51.0    46.0   

          R      H    2B   3B  ...    SH   SF   GDP    SB   CS     BA    OBP  \
0      

In [38]:
def predict_stats(player_name):
    # Filter data for the player
    player_data = grouped_data[grouped_data['Name'] == player_name]
    
    # Get the oldest age for the player
    oldest_age = player_data['Age'].min().astype(int)
    
    # Get SB and CS data for the player
    sb_cs_data = player_data[['SB', 'CS']]
    sb_cs_data.index = pd.date_range(start='2015-03-01', end='2022-12-31', freq='A')
    sb_cs_data = sb_cs_data.resample('D').interpolate()
    
    # Check if the time series is stationary
    result = adfuller(sb_cs_data['SB'])
    print(f'ADF Statistic for SB: {result[0]}')
    print(f'p-value for SB: {result[1]}')
    if result[1] <= 0.05:
        print("SB time series is stationary.")
    else:
        print("SB time series is not stationary.")
        
    result = adfuller(sb_cs_data['CS'])
    print(f'ADF Statistic for CS: {result[0]}')
    print(f'p-value for CS: {result[1]}')
    if result[1] <= 0.05:
        print("CS time series is stationary.")
    else:
        print("CS time series is not stationary.")
    
    # Plot ACF and PACF to determine AR and MA terms
    plot_acf(sb_cs_data['SB'], lags=40)
    plot_pacf(sb_cs_data['SB'], lags=40)
    plot_acf(sb_cs_data['CS'], lags=40)
    plot_pacf(sb_cs_data['CS'], lags=40)
    
    # Train ARIMA model
    model = ARIMA(sb_cs_data, order=(1,0,0))
    model = model.fit()
    
    # Check if player data is available
    if player_data.empty:
    print(f"No data found for player {player_name}.")
    else:
    # Predict stats for 2023
    forecast = model.predict(start=pd.to_datetime('2023-01-01'), end=pd.to_datetime('2023-12-31'), typ='levels')
    print(f'Forecasted SB for 2023: {forecast.sum()}')


IndentationError: expected an indented block (3254645267.py, line 42)

In [35]:
print(f'Forecasted SB for 2023: {forecast.sum()}')

NameError: name 'forecast' is not defined

In [33]:
# Example usage
player_name = input("Enter player name: ")
predict_stats(player_name)

Enter player name: Zack Short


AttributeError: 'float' object has no attribute 'astype'

In [40]:
import pandas as pd
import numpy as np
from pybaseball import batting_stats_range
from pybaseball import cache
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

cache.enable()

# Get batting stats for specified time range from PyBaseball
data = batting_stats_range(start_dt='2015-03-01', end_dt='2022-12-31')

# Group data by Name
grouped_data = data.groupby(by='Name').mean()
grouped_data.reset_index(inplace=True)
print(grouped_data)

def predict_stats(player_name):
    # Filter data for the player
    player_data = grouped_data[grouped_data['Name'] == player_name]
    
    # Check if player data is available
    if player_data.empty:
        print(f"No data found for player {player_name}.")
        return None
    
    # Get the oldest age for the player
    oldest_age = player_data['Age'].min().astype(int)
    
    # Get SB data for the player
    sb_data = player_data[['SB']]
    sb_data.index = pd.date_range(start='2015-03-01', end='2022-12-31', freq='A')
    sb_data = sb_data.resample('D').interpolate()
    
    # Check if the time series is stationary
    result = adfuller(sb_data['SB'])
    print(f'ADF Statistic: {result[0]}')
    print(f'p-value: {result[1]}')
    if result[1] <= 0.05:
        print("Time series is stationary.")
    else:
        print("Time series is not stationary.")
    
    # Plot ACF and PACF to determine AR and MA terms
    plot_acf(sb_data['SB'], lags=40)
    plot_pacf(sb_data['SB'], lags=40)
    
    # Train ARIMA model
    model = ARIMA(sb_data, order=(1,0,0))
    model = model.fit()
    
    # Predict stats for 2023
    forecast = model.predict(start=pd.to_datetime('2023-01-01'), end=pd.to_datetime('2023-12-31'), typ='levels')
    print(f'Forecasted SB for 2023: {forecast.sum()}')
    
# Example usage
player_name = input("Enter player name: ")
predict_stats(player_name)

C:\Users\Wyatt Baldwin\anaconda3\lib\site-packages\pybaseball\league_batting_stats.py:70: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  table = table.drop('', 1)


                               Name   Age   #days      G      PA      AB  \
0                      A.J. Burnett  38.0  2682.0   24.0    52.0    42.0   
1                         A.J. Cole  26.0  1761.0   19.0    30.0    26.0   
2                        A.J. Ellis  37.0  1589.0  240.0   759.0   646.0   
3                      A.J. Griffin  29.0  2007.0    3.0     8.0     7.0   
4                      A.J. Jimenez  27.0  1953.0    7.0    13.0    12.0   
...                             ...   ...     ...    ...     ...     ...   
2383                   Zack Littell  25.0   505.0    4.0     4.0     4.0   
2384                     Zack Short  27.0   159.0   67.0   197.0   165.0   
2385                   Zack Wheeler  31.0   495.0  102.0   226.0   208.0   
2386      \xc3\x81ngel Pag\xc3\xa1n  34.0  2317.0  262.0  1093.0  1006.0   
2387  \xc3\x93scar Hern\xc3\xa1ndez  26.0   918.0   25.0    51.0    46.0   

          R      H    2B   3B  ...    SH   SF   GDP    SB   CS     BA    OBP  \
0      

AttributeError: 'float' object has no attribute 'astype'